In [2]:
# import libraries
import sys
import os
import json
import yaml
import pandas as pd
import matplotlib.pylab as plt
# import modules with the functionalities offered by CompositionSpace
from compositionspace.get_gitrepo_commit import get_repo_last_commit
from compositionspace.preparation import ProcessPreparation
from compositionspace.segmentation import ProcessSegmentation
from compositionspace.clustering import ProcessClustering

In [3]:
# if desired inspect your environment
# ! pip list
print(f"Executing compositionspace in the following working directory: {os.getcwd()}")

Executing compositionspace in the following working directory /home/kaiobach/Research/hu_hu_hu/sprint19/compspace/CompositionSpace


In [6]:
MY_PROCESSED_DATA_PATH = f"{os.getcwd()}"

data = ProcessPreparation(f"{MY_PROCESSED_DATA_PATH}/experiment_params.yaml")
data.get_big_slices()
data.get_voxels()
data.calculate_voxel_composition()

Reading files:   0%|          | 0/3 [00:00<?, ?it/s]

Creating chunks:   0%|          | 0/10 [00:00<?, ?it/s]

Getting Voxels:   0%|          | 0/10 [00:00<?, ?it/s]

Calculating voxel composition:   0%|          | 0/8846 [00:00<?, ?it/s]

In [ ]:
data.chunk_files

In [ ]:
data.voxel_files

In [ ]:
data.voxel_ratio_file

In [ ]:
comps = CompositionClustering("experiment_params.yaml")
res = comps.get_PCA_cumsum(data.voxel_ratio_file, data.voxel_files[0])

In [ ]:
res = comps.get_bics_minimization(data.voxel_ratio_file, data.voxel_files[0])

In [ ]:
comps.get_composition_clusters(data.voxel_ratio_file, data.voxel_files[0])

In [ ]:
comps.generate_plots()

In [ ]:
pdata = DataPostprocess("experiment_params.yaml")
print(f"comps.voxel_centroid_output_file {comps.voxel_centroid_output_file}")

In [ ]:
## Inspect that vox_centroid_file contains indeed just copies

In [ ]:
import h5py
import numpy as np
h5r = h5py.File(comps.voxel_centroid_output_file, "r")
xyz_0 = h5r["0/0"][:, :]
xyz_1 = h5r["1/1"][:, :]
xyz_2 = h5r["2/2"][:, :]
h5r.close()
visited = np.zeros((len(xyz_2),), bool)
print(f"xyz_0 {np.shape(xyz_0)}")
print(f"xyz_1 {np.shape(xyz_1)}")
print(f"xyz_2 {np.shape(xyz_2)}")
print(f"visited {np.shape(visited)}")
print(visited)
max_diff = 0.
if False is True:
    for i in np.arange(0, np.shape(xyz_0)[0]):
        idx = int(xyz_0[i, 3])
        if visited[idx] is not True:
            visited[idx] = True
            sqr_diff = (xyz_2[idx, 0] - xyz_0[i, 0])**2 + (xyz_2[idx, 1] - xyz_0[i, 1])**2 + (xyz_2[idx, 2] - xyz_0[i, 2])**2
            if sqr_diff >= max_diff:
                max_diff = sqr_diff
            # print(f"idx {idx} sqr_diff {format(sqr_diff, '.60g')}")
        else:
            print(f"idx {idx} visited already")
    print(max_diff)
    print(np.sum(visited))
for i in np.arange(0, np.shape(xyz_1)[0]):
    idx = int(xyz_1[i, 3])
    if visited[idx] is not True:
        visited[idx] = True
        sqr_diff = (xyz_2[idx, 0] - xyz_1[i, 0])**2 + (xyz_2[idx, 1] - xyz_1[i, 1])**2 + (xyz_2[idx, 2] - xyz_1[i, 2])**2
        if sqr_diff >= max_diff:
            max_diff = sqr_diff
        # print(f"idx {idx} sqr_diff {format(sqr_diff, '.60g')}")
    else:
        print(f"idx {idx} visited already")
print(max_diff)
print(np.sum(visited))

## Conclusion: Indeed the vox_centroid_file contains unnecessary copies of positions that could be replaced with a phase_identifier array with low entropy, i.e. high compressibility

In [ ]:
from sklearn.cluster import DBSCAN

eps = 3  # self.params["ml_models"]["DBScan"]["eps"]
min_samples = 5  # self.params["ml_models"]["DBScan"]["min_samples"]

Df_centroids_no_files, Df_centroids, Phase_columns = pdata.get_post_centroids(comps.voxel_centroid_output_file, 1)
print(f"df_centroids_no_files {Df_centroids_no_files}")
print(f"Df_centroids {Df_centroids}")
print(f"Phase_columns {Phase_columns}")

db = DBSCAN(eps=eps, min_samples=min_samples).fit(Df_centroids_no_files.values)  # eps=5., min_samples= 35
print(np.unique(db.core_sample_indices_))
core_samples_mask = np.zeros_like(db.labels_, dtype=bool)
core_samples_mask[db.core_sample_indices_] = True
labels = db.labels_

In [ ]:
pdata.DBSCAN_clustering(comps.voxel_centroid_output_file, cluster_id = 0,
                        plot=True, plot3d=True, save=True)